In [15]:
from sklearn.model_selection import train_test_split

In [16]:
import pandas as pd
df = pd.read_csv('final_dataset.csv')
pd.set_option('display.max_columns', None)
df.head()

,Unnamed: 0,review_date,is_spoiler,rating_x,movie_id_y,duration,rating_y,release_date,review_without_stopwords,description_without_stopwords,review_summary_without_stopwords,plot_summary_without_stopwords,Action,Adventure,Animation,Biography,Comedy,Crime,Drama,Family,Fantasy,Film-Noir,History,Horror,Music,Musical,Mystery,Romance,Sci-Fi,Sport,Thriller,War,Western
0,0,2006,True,10,tt0111161,142,9.3,1994,"In Oscar year, Shawshank Redemption (written d...","In 1947, Andy Dufresne (Tim Robbins), banker M...",A classic piece unforgettable film-making.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,1,2000,True,10,tt0111161,142,9.3,1994,The Shawshank Redemption without doubt one bri...,"In 1947, Andy Dufresne (Tim Robbins), banker M...",Simply amazing. The best film 90's.,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,2001,True,8,tt0111161,142,9.3,1994,"I believe film best story ever told film, I'm ...","In 1947, Andy Dufresne (Tim Robbins), banker M...",The best story ever told film,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,3,2002,True,10,tt0111161,142,9.3,1994,"**Yes, SPOILERS here**This film emotional impa...","In 1947, Andy Dufresne (Tim Robbins), banker M...",Busy dying busy living?,Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,4,2004,True,8,tt0111161,142,9.3,1994,At heart extraordinary movie brilliant indelib...,"In 1947, Andy Dufresne (Tim Robbins), banker M...","Great story, wondrously told acted",Chronicles experiences formerly successful ban...,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [17]:
import re


def clean_text(text):
    """
    Applies some pre-processing on the given text.

    Steps :
    - Removing HTML tags
    - Removing punctuation
    - Lowering text
    """
    
    # remove HTML tags
    text = re.sub(r'<.*?>', '', text)
    
    # remove the characters [\], ['] and ["]
    text = re.sub(r"\\", "", text)    
    text = re.sub(r"\'", "", text)    
    text = re.sub(r"\"", "", text)    
    
    # convert text to lowercase
    text = text.strip().lower()
    
    # replace punctuation characters with spaces
    filters='!"\'#$%&()*+,-./:;<=>?@[\\]^_`{|}~\t\n'
    translate_dict = dict((c, " ") for c in filters)
    translate_map = str.maketrans(translate_dict)
    text = text.translate(translate_map)

    return text


In [40]:
df["review_without_stopwords"] = df["review_without_stopwords"].astype(str)

In [41]:
X = df["review_without_stopwords"].apply(lambda x: clean_text(x))
y = df['is_spoiler']
y = y.apply(lambda x: 1 if x else 0)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [10]:
y_train

64659     0
234573    0
24086     1
483424    0
342693    0
         ..
110268    0
259178    0
365838    1
131932    0
121958    1
Name: is_spoiler, Length: 430429, dtype: int64

In [42]:
df_train = pd.concat([X_train,y_train],axis=1)
df_train = df_train.rename(columns={"review_without_stopwords": "text", "is_spoiler": "labels"})
df_train

,text,labels
64659,i think raging bull one best films ever made ...,0
234573,if want movie insanely excellent acting skills...,0
24086,as jiddu krishnamurti said it measure health ...,1
483424,movie sequels films usually poor reputation d...,0
342693,in part one neo destroys agent smith smith gai...,0
...,...,...
110268,blade runner 2049 surprising squeal manages im...,0
259178,my wife excited saw trailer the accountant fi...,0
365838,when i saw movie i really care about johnny d...,1
131932,i love original movies ie the book eli watchm...,0


In [43]:
df_test = pd.concat([X_test,y_test],axis=1)
df_test = df_test.rename(columns={"review_without_stopwords": "text", "is_spoiler": "labels"})
df_test

,text,labels
389140,i wonder film would judged without spielbergs ...,0
235482,opening scene girl driving 30 seconds late...,1
294615,moulin rouge kind film makes aware true possib...,0
336345,not sure sticking craw reviewers lxg it nearl...,0
378675,peter jackson gave movie new twist i ve seen ...,0
...,...,...
126785,first let say liked one better first iron man ...,1
126711,the film slow moving also outstanding histori...,0
148034,johnny depp lost bit pirate charm newest insta...,0
103883,in mood love directed wong kar wai i feel re...,0


In [22]:
from simpletransformers.classification import ClassificationModel


model = ClassificationModel('roberta', 'roberta-base',use_cuda=False)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

In [63]:
model.train_model(df_train.head(500))

(63, 0.60060044555437)

In [69]:
y_train.head(500).value_counts()

0    363
1    137
Name: is_spoiler, dtype: int64

In [66]:
result, model_outputs, wrong_predictions = model.eval_model(df_test.head(100))

/home/sahilvaidya/anaconda3/lib/python3.7/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in double_scalars
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [67]:
print(result)

{'mcc': 0.0, 'tp': 0, 'tn': 73, 'fp': 0, 'fn': 27, 'auroc': 0.6910197869101979, 'auprc': 0.5301110537300806, 'eval_loss': 0.5995816748875838}


In [68]:
print(model_outputs)

[[ 0.43835917 -0.39002618]
 [ 0.43854895 -0.39111289]
 [ 0.43840143 -0.38484836]
 [ 0.44126773 -0.38385046]
 [ 0.44398668 -0.39088172]
 [ 0.43391487 -0.39459562]
 [ 0.44226867 -0.36711916]
 [ 0.4444381  -0.38498652]
 [ 0.42655018 -0.36995205]
 [ 0.45215586 -0.39471012]
 [ 0.445609   -0.36301377]
 [ 0.44631696 -0.37827119]
 [ 0.44437012 -0.36992759]
 [ 0.46215901 -0.37859881]
 [ 0.44138047 -0.37762073]
 [ 0.45333982 -0.37848061]
 [ 0.4444986  -0.38663763]
 [ 0.42970714 -0.38885415]
 [ 0.45548159 -0.37991038]
 [ 0.44304672 -0.38547635]
 [ 0.4430137  -0.37784475]
 [ 0.43600765 -0.38655251]
 [ 0.43991625 -0.36534739]
 [ 0.44365123 -0.38048053]
 [ 0.43496838 -0.38947439]
 [ 0.4494532  -0.37649342]
 [ 0.436322   -0.37347513]
 [ 0.42250782 -0.38146162]
 [ 0.43258864 -0.39109504]
 [ 0.44181591 -0.37248933]
 [ 0.436598   -0.37010092]
 [ 0.4248873  -0.37691247]
 [ 0.42870227 -0.37553534]
 [ 0.43475726 -0.39241755]
 [ 0.45185059 -0.38612202]
 [ 0.43328834 -0.37164614]
 [ 0.44768545 -0.39522409]
 

In [61]:
print(y_test.head(50).tolist())

[0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1]
